In [10]:
## The first cell is the API from DragonChain

In [1]:
import json
import requests
import pandas as pd

from pygments import highlight
from pygments.lexers.data import JsonLexer
from pygments.formatters.terminal import TerminalFormatter


# Each lair has it's own stream of content and every user has their own
# main stream that consist of lairs they're subscribed to and other users
# that they follow.
#
# Unauthenticated requests will not return any content for "follow" users
# and only returns content for lairs that have been configured for unauthenticated
# requests. However, you can get the streams for any public lair directly. There
# is also an example API call below for fetching all the public lairs.
#
# Here is an example of two pieces of content that is returned by the steam API.
# First one is a post and the second is a reply to that post (but can be a reply to a reply)
#
# These examples come from a post I made in the Maker lair. Here is the link to post on Den
# https://den.social/l/maker/ezNPiC3Edg/hmm-this-looked-a-lot-bigger-in-the-picture/
#
# {
#   "id": "ezNPiC3Edg",
#   "created": 1611372847,
#   "createdNano": 1611372847475916000,
#   "modified": 1611372847,
#   "modifiedNano": 1611372847475916000,
#
#       # contentType can either be 'post' or 'reply'
#   "contentType": "post",
#
#       # 'content' contains the render HTML (default) for the post/reply
#       # The markdown version can be fetched instead (see query strings below) if need be
#       # But note that the server rendered HTML is sanitized for malicious scripts
#       # and some posts are only HTML and not markdown
#   "content": "...",
#   "dcTransactionId": "8f8184e3-38c1-47d1-824f-3e3bb92f64a7",
#   "renderType": "",
#   "author": {
#     "id": "b6a473cc086f424c95c2e21917a352e9",
#     "registered": 1574276945,
#     "displayName": "Zeth",
#     "description": "",
#     "flagImgId": "67d3d1ad28694757978250e4cccf9318",
#     "avatarImgId": "",
#     "siteReputation": 100,
#     "lairReputation": 100,
#     "isBot": false
#   },
#   "authorBanned": false,
#   "lair": {
#     "id": "4fcac97e2a7b4d3a8f33d8036dae8ed2",
#     "name": "maker",
#     "displayName": "Maker"
#   },
#   "stats": {
#     "score": 521,
#     "upVoteScore": 521,
#     "downVoteScore": 0,
#     "upVoters": 3,
#     "downVoters": 0,
#     "views": 35,
#     "replyCount": 1, # direct replies only (bug)
#     "trendingScoreMultiplier": 0.0001,
#     "weightedScore": 0.052,
#     "mtrBoost": "0"
#   },
#   "published": true,
#   "sticky": false,
#   "edited": false,
#   "url": "/l/maker/ezNPiC3Edg/hmm-this-looked-a-lot-bigger-in-the-picture/",
#
#       # Deprecated: ignore this field and use 'media' instead
#   "image": {
#     "id": "baa308fc8c94426fad5c1b1dc78fde29"
#   },
#   "media": [
#     {
#       # Images can be fetched by added this ID to the end of this URL
#       # https://media-den.dragonchain.com/content/images/<image id>
#       "image": "baa308fc8c94426fad5c1b1dc78fde29"
#     }
#   ],
#   "depth": 0,
#   "blocked": false,
#   "deleted": false,
#   "deletedBy": "",
#   "title": "Hmm, This Looked A Lot Bigger In The Picture :("
# }
#
# Reply to the post above ^
#
# The main differance is that replies include a short version of the post's JSON under `replyContent`
# Replies also do not have `title`s
#
# The `content` fields inside of `replyContent` is also only a summary and not the full content
#
# {
#   "id": "V31G2sJUep",
#   "created": 1611372927,
#   "createdNano": 1611372927827388000,
#   "modified": 1611372927,
#   "modifiedNano": 1611372927827388000,
#   "contentType": "reply",
#   "content": "...",
#   "dcTransactionId": "1490eee8-d3b7-4962-896d-881291602104",
#   "renderType": "",
#   "author": {
#     "id": "b6a473cc086f424c95c2e21917a352e9",
#     "registered": 1574276945,
#     "displayName": "Zeth",
#     "description": "",
#     "flagImgId": "67d3d1ad28694757978250e4cccf9318",
#     "avatarImgId": "",
#     "siteReputation": 100,
#     "lairReputation": 100,
#     "isBot": false
#   },
#   "authorBanned": false,
#   "lair": {
#     "id": "4fcac97e2a7b4d3a8f33d8036dae8ed2",
#     "name": "maker",
#     "displayName": "Maker"
#   },
#   "stats": {
#     "score": 21,
#     "upVoteScore": 21,
#     "downVoteScore": 0,
#     "upVoters": 2,
#     "downVoters": 0,
#     "views": 1,
#     "replyCount": 2,
#     "trendingScoreMultiplier": 0.0001,
#     "weightedScore": 0.002,
#     "mtrBoost": "0"
#   },
#   "published": true,
#   "sticky": false,
#   "edited": false,
#   "url": "/l/maker/ezNPiC3Edg/hmm-this-looked-a-lot-bigger-in-the-picture/?r=V31G2sJUep",
#   "depth": 1, # how deep this reply is in the thread
#   "blocked": false,
#   "deleted": false,
#   "deletedBy": "",
#   "replyContent": {
#     "id": "ezNPiC3Edg",
#     "created": 1611372847,
#     "modified": 1611372847,
#     "contentType": "post",
#     "content": "...",
#     "dcTransactionId": "8f8184e3-38c1-47d1-824f-3e3bb92f64a7",
#     "title": "Hmm, This Looked A Lot Bigger In The Picture :(",
#     "renderType": "html",
#     "author": {
#       "id": "b6a473cc086f424c95c2e21917a352e9",
#       "registered": 1574276945,
#       "displayName": "Zeth",
#       "description": "",
#       "flagImgId": "67d3d1ad28694757978250e4cccf9318",
#       "avatarImgId": "",
#       "siteReputation": 100,
#       "lairReputation": 100,
#       "isBot": false
#     },
#     "authorBanned": false,
#     "image": {
#       "id": "baa308fc8c94426fad5c1b1dc78fde29"
#     },
#     "media": [
#       {
#         "image": "baa308fc8c94426fad5c1b1dc78fde29"
#       }
#     ],
#     "postId": "ezNPiC3Edg",
#     "blocked": false,
#     "deleted": false,
#     "deletedBy": ""
#   }
# }

# Base URL for Den's production API
BASE_API_URL = 'https://api.den.social'

# Please change this to something unique for your scripts.
# That will allow me to filter logs if you need me to review something
USER_AGENT = 'den-api-example-script'

# Each of these params are a query string options
#
# There are two methods to page through a stream. Either using the limit/offset
# method or the 'limit/before or after' method.
#
# Limit/offset is your standard paging method. To use the 'limit/before or after', you pass the
# 'createdNano' timestamp of a piece of content to either 'before' or 'after' to get the number
# of content given by 'limit' (offset ignored) either created before or created after that timestamp
#
# sort_by: default is 'created'. Pass 'trending' for get the stream's trending content.
#
# content_type: options are 'None' to get both post and replies.
#                           'post' to get only posts
#                           'replies' to get only replies
#
# post_id: returns only replies for that given post
# lair_id: returns only content for the given lair
# lair_name: returns only content for the given lair (NOTE: not the lair displayName)
# author_id: returns only content for the given user account
# markdown: returns the markdown version of the content's text (not all content has markdown)
def get_stream(limit=None, offset=None, before=None, after=None, sort_by=None, content_type=None,
               post_id=None, lair_id=None, lair_name=None, author_id=None, markdown=False):

    query_string = {}

    if limit is not None:
        query_string['limit'] = limit
    if offset is not None:
        query_string['offset'] = offset
    if sort_by is not None:
        query_string['sortBy'] = sort_by
    if before is not None:
        query_string['before'] = before
    if after is not None:
        query_string['after'] = after
    if content_type is not None:
        query_string['contentType'] = content_type
    if post_id is not None:
        query_string['postId'] = post_id
    if lair_id is not None:
        query_string['lairId'] = lair_id
    if lair_name is not None:
        query_string['lairName'] = lair_name
    if author_id is not None:
        query_string['authorId'] = author_id
    if markdown:
        query_string['markdown'] = 'true'

    headers = {
        'ContentType': 'application/json',
        'User-Agent': USER_AGENT,
    }

    resp = requests.get(BASE_API_URL + '/api/v1/content', params=query_string, headers=headers)
    return resp.status_code, resp.json()


# We only have about 50 public lairs currently. I would just grab them all in one go.
def get_lairs(limit=300, offset=0):
    params = {'limit': limit, 'offset': offset}
    headers = {
        'ContentType': 'application/json',
        'User-Agent': USER_AGENT,
    }
    resp = requests.get(BASE_API_URL + '/api/v1/lairs', params=params, headers=headers)
    return resp.status_code, resp.json()


def print_resp(status_code, resp, quit_on_error=True):
    print('Stats:', status_code)
    print('Response')

    if resp is None:
        print('No Data!')
        if quit_on_error and status_code >= 400:
            exit()
        return

    if type(resp) != list and 'error' in resp.keys():
        print('Error: ', resp['error'])
    else:
        print_json = json.dumps(resp, indent=True)
        print(highlight(print_json, JsonLexer(), TerminalFormatter()))

    if quit_on_error and status_code >= 400:
        exit()


def main_L():
#     Get all lairs
    status_code, resp = get_lairs()
    return  resp

    # Get posts and replies for main unauthenticated stream
#     status_code, resp = get_stream(limit=15, offset=0)
#     return status_code, resp

    # Get only posts for main unauthenticated stream
    # status_code, resp = get_stream(limit=15, offset=0, content_type='post')
    # print_resp(status_code, resp)

    # Get posts and replies for Maker lair stream by id
    # status_code, resp = get_stream(limit=15, offset=0, lair_id='4fcac97e2a7b4d3a8f33d8036dae8ed2')
    # print_resp(status_code, resp)

#     Get posts and replies for Maker lair stream by name
#     status_code, resp = get_stream(limit=500, offset=0)
#     return status_code, resp

    # Get all replies for post 'ezNPiC3Edg' thread
    # status_code, resp = get_stream(limit=100, offset=0, post_id='ezNPiC3Edg', content_type='reply')
    # print_resp(status_code, resp)
    
def main_p(x):
    # Get posts and replies for Maker lair stream by id
    status_code, resp = get_stream(limit=100, offset=0, lair_id=x)
    return resp


if __name__ == '__main__':
    main_L()


First we have to store the fetched data

In [2]:
Lairs = main_L()

In [3]:
Lairs['lairs']

[{'id': '72e69a5313b9434f9fd378d5087bd3d9',
  'created': 1584034581,
  'modified': 1588349018,
  'name': 'covid19',
  'avatarImgId': 'bb3917a3c44f4d92a60ed21245678dae',
  'bannerImgId': '9a020bc0e0ca42b4867a6f8f9dd7701d',
  'backgroundImgId': '',
  'url': '',
  'displayName': 'Coronavirus',
  'description': 'Please be cognizant of where you comment. Please try to keep everything under the correct date if possible. If you have another area you would like tracked please let us know, or you can post on your own. Please source all figures. ',
  'subscribed': False,
  'isPrivate': False,
  'monetized': True,
  'owners': ['3baf141864f24738831da145235a7828',
   '714427dcb5304f1fa47de24d988cc367',
   'bd7bdcdd7bef4225966e19175dae8a75',
   'fe523ac027bb4c9fa612d4f2997ceece'],
  'admins': ['fe523ac027bb4c9fa612d4f2997ceece'],
  'lairPermissions': {'system': 'simple',
   'post': {'enabled': True,
    'time': 0,
    'siteRep': 0,
    'lairRep': 0,
    'repThreshold': 13},
   'reply': {'enabled': T

Checkout all the Lairs 

In [4]:
for i in Lairs['lairs']:
    print(i['displayName'])



Coronavirus


CrowsNest
AMA
The Bitcoin Street Journal
Smaugust
Satoshi Sean & The Irishman Dragon’s Hoard
Dragonchain Ufficiale Italiano
Dragonchain Nederland Officieel
Q
공식 드래곤체인 한국어
Memes
Welcome To The Matrix
Dragonchain Officiel Français
Introduce Yourself
Eternal Archive
Let’s Hash it Out
Red Pills
Medical/ safety 

Dragonchain Oficial Português
Sports
Maker
Dance
Pets
Music
Dragonchain Oficial Español
Christianity
Creature Comforts (Food & Beverage)
Politics
Den
Den Governance 2021
{ Code }
Bored
All Things Cannabis
Den Support
Art
ISLAM
Comunidad Hispana
Gaming
Beaxy Exchange
Health Fitness and Nutrition
Today I Learned
Lair Creation Process (LCP)
LBTY
Outdoors
Hacker Noon
公式ドラゴンチェーン日本語


#### This is a dataframe of all of the content from Lairs. This contains information about the lairs. 

In [5]:
#List for content in Lairs
name = []
display_name = []
id_code = []
created = []
is_monetized = []
owners = []
is_private = []
reputation = []
description = []
avatarImgId = []
url = []
description = []
subscribed = []
isPrivate = []
monetized = []
owners = []
admins = []
lairPermissions = []

#Appends all content to lists
for i in Lairs['lairs']:
    name.append(i['name'])
    display_name.append(i['displayName'])
    id_code.append(i['id'])
    created.append(i['created'])
    is_monetized.append(i['monetized'])
    owners.append(i['owners'])
    is_private.append(i['isPrivate'])
    reputation.append(i['reputation'])
    description.append(i['description'])
    avatarImgId.append(i['avatarImgId'])
    url.append(i['url'])
    description.append(i['description'])
    subscribed.append(i['subscribed'])
    isPrivate.append(i['isPrivate'])
    monetized.append(i['monetized'])
    owners.append(i['owners'])
    admins.append(i['admins'])
    lairPermissions.append(i['lairPermissions'])

In [6]:
# All created lists of Lair information is created in to dataframe constructer after zipping 
lair_data = pd.DataFrame(list(zip(display_name,name, id_code, created,owners,reputation,description,avatarImgId,url,description,subscribed,isPrivate,monetized,owners,admins,lairPermissions)), 
               columns =['Display Name','Name','ID','Created','Owners','Reputation','Description','avatarImgId','url','description','subscribed','isPrivate','monetized','owners','admins','lairPermissions'],index = name) 
lair_data

,Display Name,Name,ID,Created,Owners,Reputation,Description,avatarImgId,url,description,subscribed,isPrivate,monetized,owners,admins,lairPermissions
covid19,Coronavirus,covid19,72e69a5313b9434f9fd378d5087bd3d9,1584034581,"[3baf141864f24738831da145235a7828, 714427dcb53...",0,Please be cognizant of where you comment. Plea...,bb3917a3c44f4d92a60ed21245678dae,,Please be cognizant of where you comment. Plea...,False,False,True,"[3baf141864f24738831da145235a7828, 714427dcb53...",[fe523ac027bb4c9fa612d4f2997ceece],"{'system': 'simple', 'post': {'enabled': True,..."
Cryptocurrency,,Cryptocurrency,3ce33323cd4c42b584260af65d8982a8,1574280058,"[3baf141864f24738831da145235a7828, 714427dcb53...",0,Please be cognizant of where you comment. Plea...,27ac4b0bd4974820970d66ccd5654659,,Please be cognizant of where you comment. Plea...,False,False,True,"[3baf141864f24738831da145235a7828, 714427dcb53...",[bd7bdcdd7bef4225966e19175dae8a75],"{'system': 'simple', 'post': {'enabled': True,..."
TechNews,,TechNews,95d80511ecbc443ea8dda487762fe106,1574280059,[bd7bdcdd7bef4225966e19175dae8a75],0,"Open to discussion of any cryptocurrency, bloc...",a99901adb0824741a52c4800a252e30c,,"Open to discussion of any cryptocurrency, bloc...",False,False,True,[bd7bdcdd7bef4225966e19175dae8a75],[bd7bdcdd7bef4225966e19175dae8a75],"{'system': 'simple', 'post': {'enabled': True,..."
CrowsNest,CrowsNest,CrowsNest,9845e14a1d564348ab1edc6f3fa915de,1589559808,[bd7bdcdd7bef4225966e19175dae8a75],0,"Open to discussion of any cryptocurrency, bloc...",d4de8a7302ff4a5d8412544a03d8b91c,http://www.youtube.com/c/CryptoCrowOfficial,"Open to discussion of any cryptocurrency, bloc...",False,False,True,[bd7bdcdd7bef4225966e19175dae8a75],[6f4a3e3338444011afc4ccd1c7031ba4],"{'system': 'simple', 'post': {'enabled': True,..."
ama,AMA,ama,02b1969dc52b46eb8894b1b802b16cba,1578345779,[bd7bdcdd7bef4225966e19175dae8a75],0,,5333b27e3be8422e966fb62f62693e52,,,False,False,True,[bd7bdcdd7bef4225966e19175dae8a75],[bd7bdcdd7bef4225966e19175dae8a75],"{'system': 'simple', 'post': {'enabled': False..."
TheBitcoinStreetJournal,The Bitcoin Street Journal,TheBitcoinStreetJournal,032e7b05a35243a2995bcf9bcbe25b06,1590520889,[bd7bdcdd7bef4225966e19175dae8a75],0,,d9c29bbc43d24428a836ceabcac5fc8f,https://thebitcoinstreetjournal.com,,False,False,True,[bd7bdcdd7bef4225966e19175dae8a75],[606baa2b23ad4026943eaa4e2ac8adc4],"{'system': 'simple', 'post': {'enabled': True,..."
smaugust,Smaugust,smaugust,8b30e7f81e424463867ae74b0f20bf75,1592257171,[6f4a3e3338444011afc4ccd1c7031ba4],0,"Community home of the Crypto Crow, known on Yo...",3533ffd4196a4bf7ab822a6a7c2a7ae1,,"Community home of the Crypto Crow, known on Yo...",False,False,True,[6f4a3e3338444011afc4ccd1c7031ba4],[ea4d8ba4ed0940d4b36f8ac9c1702437],"{'system': 'simple', 'post': {'enabled': False..."
Dragonshoard,Satoshi Sean & The Irishman Dragon’s Hoard,Dragonshoard,7c78640fc3a240f580c89546b6c8e10e,1591646702,[6f4a3e3338444011afc4ccd1c7031ba4],0,"Community home of the Crypto Crow, known on Yo...",,,"Community home of the Crypto Crow, known on Yo...",False,False,True,[6f4a3e3338444011afc4ccd1c7031ba4],[df4cf9d3ad6d4ad0b4511371acb3d6fb],"{'system': 'simple', 'post': {'enabled': True,..."
dragonchain-official-it,Dragonchain Ufficiale Italiano,dragonchain-official-it,79e5e63687404855bd745b3faa4c8c50,1593640805,[bd7bdcdd7bef4225966e19175dae8a75],0,This Lair is for Ask Me Anything (AMA) sessions.,c3673309bda44dcbb0393026809a4676,,This Lair is for Ask Me Anything (AMA) sessions.,False,False,True,[bd7bdcdd7bef4225966e19175dae8a75],[7eba708abee046fd9d0c9b40a8bf830a],"{'system': 'simple', 'post': {'enabled': True,..."
dragonchain-official-nl,Dragonchain Nederland Officieel,dragonchain-official-nl,bd9f203ec3e847febf474269f3f90913,1593640840,[bd7bdcdd7bef4225966e19175dae8a75],0,This Lair is for Ask Me Anything (AMA) sessions.,d2f0be66d0d04a3a9cbec4b107d64dd8,,This Lair is for Ask Me Anything (AMA) sessions.,False,False,True,[bd7bdcdd7bef4225966e19175dae8a75],[323147a6661446038

In [7]:
lair_data['ID']

covid19                    72e69a5313b9434f9fd378d5087bd3d9
Cryptocurrency             3ce33323cd4c42b584260af65d8982a8
TechNews                   95d80511ecbc443ea8dda487762fe106
CrowsNest                  9845e14a1d564348ab1edc6f3fa915de
ama                        02b1969dc52b46eb8894b1b802b16cba
TheBitcoinStreetJournal    032e7b05a35243a2995bcf9bcbe25b06
smaugust                   8b30e7f81e424463867ae74b0f20bf75
Dragonshoard               7c78640fc3a240f580c89546b6c8e10e
dragonchain-official-it    79e5e63687404855bd745b3faa4c8c50
dragonchain-official-nl    bd9f203ec3e847febf474269f3f90913
qanon                      f330b68629c047b195e5296264f270af
dragonchain-official-kr    eef0a69a24704a58912ebdc25f0e4848
memes                      4e946f6f071f425ebc26b1e86c3678e6
welcome-to-the-matrix      3b9f0b6e23264782bbd4e3e27749ca87
dragonchain-official-fr    2535f1b8ed5d4d45be5af9042c81f7a3
hello                      196e06ad3576458fb93f06be0a0d1e26
eternal-archive            cfed09ff113b4

In [8]:
#Looking at the first lair Name 
lair_data['Name'][0]

'covid19'

In [9]:
### Lairs_dict is going to be a dictionary. Keys are the lair names, and values are Lair IDs

In [10]:
Lairs_dict = {}
for Lair in Lairs['lairs']:
    Lairs_dict[Lair['displayName']] = Lair['id']

In [11]:
## content_by_id is a dictionary with keys are user IDs and values are lists of posts they make 

In [12]:
content_by_id = {}

for i in Lairs_dict.values():
    posts = main_p(i)

    for content in posts['contents']:
        try:
            content_by_id[content['author']['id']].append(content['content'])
        except:
            content_by_id[content['author']['id']] = []
            content_by_id[content['author']['id']].append(content['content'])    

In [13]:
content_by_id['cab6d19d22ec4bdfad4f99760d33312b']


['<p>Yeah, not masks haha.</p>',
 '<p>Wow. How is that possible?</p>',
 '<p>No joke, you just got it?</p>',
 '<p>One of those signatures is mine.</p>',
 '<p>Haha no worries. It&#39;s a public document.</p>',
 '<p>This person (1) doesn&#39;t know what he&#39;s talking about and (2) has something driving him to post constantly about us - I wonder why that is? 🤔 🤷\u200d♂️\xa0</p>\n<p>IGNORE - Your time is better spent on positive things such as your family or by spreading love in the Dragon community. 👍</p>',
 '<p>Some things to address here…\xa0</p>\n<p><a href="https://twitter.com/N0UR______/status/1252764179204698112?s=19" target="_blank" rel="nofollow noopener">https://twitter.com/N0UR______/status/1252764179204698112?s=19</a></p>',
 '<p>Not sure that I understand - pools for L1? Do you mean shared business nodes?</p>',
 '<p>All questions answered here:\xa0<a href="https://www.youtube.com/watch?feature=youtu.be&v=QKt9bBbr3zE" target="_blank" rel="nofollow noopener">https://www.youtube

In [14]:
#### Turning the orignal dicitonary readable for pandas
max_len = 0
for i in content_by_id.keys():
    if len(content_by_id[i])> max_len:
        max_len = len(content_by_id[i])
max_len

for i in content_by_id.keys():
    value_to_add = max_len-len(content_by_id[i])
    if len(content_by_id[i]) == max_len:
        continue
    for j in range(0,value_to_add):
        content_by_id[i].append('')

In [15]:
key_list = list(content_by_id.keys())
key_list[0]

'33ae5b6690d14b9985abd530b281fafb'

Example:

In [16]:
#Using the values in the key list to grab content by id
content_by_id[key_list[0]]

['<p>Self-reported speculative side effects, not side effects that were shown under controlled conditions.</p>\n<p>I&#39;m not dismissing it wholesale even though it may sound that way.\xa0 I do hold confirmed and reputably peer-reviewed science over speculative assumption.\xa0 Those cases should be investigated and the results should be published, of course.\xa0 Of there&#39;s some side effect that&#39;s weird I&#39;d like to know about it, absolutely.\xa0\xa0</p>\n<p>That database doesn&#39;t scare me away personally though.</p>',
 '<p>this is from the database curators that the other list of side effects sites, just for reference and perspective.</p>',
 '<p>meh, I&#39;m less worried about parts of a virus than a whole virus, this is basically hacking the way that viruses reproduce to instead produce just the viral spikes.  </p>\n<p>to me, it&#39;s way safer.  personal opinion though.</p>',
 '<p>I&#39;m currently in line for my 2nd Pfizer dose and thought this was an interesting comp

In [17]:
#### store everything in a Pandas DataFrame
import pandas as pd

In [18]:
df =pd.DataFrame.from_dict(content_by_id)
# df.to_csv('user_ids and their posts.csv')

In [19]:
df[list(content_by_id.keys())[0:50]].to_csv('user_ids and their posts_downsized.csv')

# Check here

In [20]:
#This takes all the data coming from the first lair which is the lair named covid19 - aka Coronavirus
data = main_p(lair_data['ID'][20])
data['count']

100

In [21]:
#This looks at just the first comment in the coronavirus lair
data['contents'][0]

{'id': 'AVPp0Xr1vv',
 'created': 1614983204,
 'createdNano': 1614983204482718000,
 'modified': 1614983204,
 'modifiedNano': 1614983204482718000,
 'contentType': 'reply',
 'content': '<p>He allegedly did that but at the end of the day everyone had a choice to buy or not.</p>',
 'dcTransactionId': '',
 'renderType': '',
 'author': {'id': '150293a5efa64a0ab86448fa59a062f3',
  'registered': 1580956682,
  'displayName': 'Crypto_Enthusiast',
  'description': "Dragonchain supporter and also anyone that supports me will also in turn get my support of NRG. I'm here for the revolution that is Blockchain!  Lets ride together and earn NRG together, on this spaceship called DC!! My Twitter handle is @Cryptoartyst.",
  'flagImgId': '7238543590e3426095581ba710db41b1',
  'avatarImgId': '8dbc832e2dab45848ce5af98e90c5057',
  'siteReputation': 95.64,
  'lairReputation': 0,
  'isBot': False},
 'authorBanned': False,
 'lair': {'id': '35e0c07f244c484db4df398f7a323b08',
  'name': 'Dragonchain',
  'displayNam

In [22]:
#Note this is not a complete list of all the information that can be pulled from the data, however I tried to grab
#the most relevant information that could be used for analysis.

#This tells you the title of the post
title = data['contents'][0]['title']

#This gives the url of the post 
url = data['contents'][0]['url']

#This tells you if it is a 'post' or 'reply'
contentType = data['contents'][0]['contentType']

#This gives the content of the comment
content = data['contents'][0]['content']

#This gives information on the author
author_id = data['contents'][0]['author']['id']
author_displayName = data['contents'][0]['author']['displayName']
author_siteRep = data['contents'][0]['author']['siteReputation']
author_lairRep = data['contents'][0]['author']['lairReputation']
author_isBot = data['contents'][0]['author']['isBot']

# This shows which lair the contents came from 
lair_id = data['contents'][0]['lair']['id']
lair_name = data['contents'][0]['lair']['name']
lair_displayName = data['contents'][0]['lair']['displayName']

#This is all the stats of that individual conmment
stats_score = data['contents'][0]['stats']['score']
stats_upVoteScore = data['contents'][0]['stats']['upVoteScore']
stats_downVoteScore = data['contents'][0]['stats']['downVoteScore']
stats_upVoters = data['contents'][0]['stats']['upVoters']
stats_views = data['contents'][0]['stats']['views']
stats_replyCount = data['contents'][0]['stats']['replyCount']
stats_trendingScoreMultiplier = data['contents'][0]['stats']['trendingScoreMultiplier']
stats_weightedScore = data['contents'][0]['stats']['weightedScore']
stats_mtrBoost = data['contents'][0]['stats']['mtrBoost']

KeyError: 'title'

In [23]:
url = []
contentType = []
content = []
author_id = []
author_displayName = []
author_siteRep = []
author_lairRep = []
author_isBot = []
lair_id = []
lair_name = []
lair_displayName = []
stats_score = []
stats_upVoteScore = []
stats_downVoteScore = []
stats_upVoters = []
stats_views = []
stats_replyCount = []
stats_trendingScoreMultiplier = []
stats_weightedScore = []
stats_mtrBoost = []

for i in range(data['count']):
    url.append(data['contents'][i]['url'])
    contentType.append(data['contents'][i]['contentType'])

    #This gives the content of the comment
    content.append(data['contents'][i]['content'])

    #This gives information on the author
    author_id.append(data['contents'][i]['author']['id'])
    author_displayName.append(data['contents'][i]['author']['displayName'])
    author_siteRep.append(data['contents'][i]['author']['siteReputation'])
    author_lairRep.append(data['contents'][i]['author']['lairReputation'])
    author_isBot.append(data['contents'][i]['author']['isBot'])

    # This shows which lair the contents came from 
    lair_id.append(data['contents'][i]['lair']['id'])
    lair_name.append(data['contents'][i]['lair']['name'])
    lair_displayName.append(data['contents'][i]['lair']['displayName'])

    #This is all the stats of that individual conmment
    stats_score.append(data['contents'][i]['stats']['score'])
    stats_upVoteScore.append(data['contents'][i]['stats']['upVoteScore'])
    stats_downVoteScore.append(data['contents'][i]['stats']['downVoteScore'])
    stats_upVoters.append(data['contents'][i]['stats']['upVoters'])
    stats_views.append(data['contents'][i]['stats']['views'])
    stats_replyCount.append(data['contents'][i]['stats']['replyCount'])
    stats_trendingScoreMultiplier.append(data['contents'][i]['stats']['trendingScoreMultiplier'])
    stats_weightedScore.append(data['contents'][i]['stats']['weightedScore'])
    stats_mtrBoost.append(data['contents'][i]['stats']['mtrBoost'])

#url,contentType,content,author_id,author_displayName,author_siteRep,author_lairRep,author_isBot,lair_id,lair_name,lair_displayName,stats_score,stats_upVoteScore,stats_downVoteScore,stats_upVoters,stats_views,stats_replyCount,stats_trendingScoreMultiplier,stats_weightedScore,stats_mtrBoost

comment_data = pd.DataFrame(list(zip(url,contentType,content,author_id,author_displayName,
                                  author_siteRep,author_lairRep,author_isBot,lair_id,lair_name,
                                  lair_displayName,stats_score,stats_upVoteScore,stats_downVoteScore,
                                  stats_upVoters,stats_views,stats_replyCount,stats_trendingScoreMultiplier,
                                  stats_weightedScore,stats_mtrBoost)),
                         columns =['url','contentType','content','author_id','author_displayName',
                                   'author_siteRep','author_lairRep','author_isBot','lair_id',
                                   'lair_name','lair_displayName','stats_score','stats_upVoteScore',
                                   'stats_downVoteScore','stats_upVoters','stats_views','stats_replyCount',
                                   'stats_trendingScoreMultiplier','stats_weightedScore','stats_mtrBoost'],
                         index = author_displayName) 

In [24]:
#So no we have a dataframe with all of this content information that came from the Coronovirus Lair
#Now, we can play around with this for analysis.
comment_data

,url,contentType,content,author_id,author_displayName,author_siteRep,author_lairRep,author_isBot,lair_id,lair_name,lair_displayName,stats_score,stats_upVoteScore,stats_downVoteScore,stats_upVoters,stats_views,stats_replyCount,stats_trendingScoreMultiplier,stats_weightedScore,stats_mtrBoost
Crypto_Enthusiast,/l/Dragonchain/HmGO6thY4k/mcafee-indictment-an...,reply,<p>He allegedly did that but at the end of the...,150293a5efa64a0ab86448fa59a062f3,Crypto_Enthusiast,95.64,0,False,35e0c07f244c484db4df398f7a323b08,Dragonchain,,0,0,0,0,0,0,20.0000,0.000,0
florijanll,/l/Dragonchain/HmGO6thY4k/mcafee-indictment-an...,reply,<p>I mean this is what you get for organizing ...,cd5d45d19a4e45e49a0c6e01c79f97cb,florijanll,99.03,0,False,35e0c07f244c484db4df398f7a323b08,Dragonchain,,55,55,0,1,0,1,20.0000,0.000,0
1.8t,/l/Dragonchain/nqK31ee6MB/any-updates-on-conso...,reply,<p>I have wondered these same questions. I ho...,cac1e219b08e40a7aa60226c7969a6e5,1.8t,97.96,0,False,35e0c07f244c484db4df398f7a323b08,Dragonchain,,5,5,0,1,1,0,20.0000,100.000,0
JR1,/l/Dragonchain/nqK31ee6MB/any-updates-on-conso...,post,<p>Today I was looking at the withdraw fees on...,55bebb1187a742208ba3b971186358b0,JR1,97.16,0,False,35e0c07f244c484db4df398f7a323b08,Dragonchain,,1459,1459,0,5,43,1,20.0000,29180.000,0
Crypto_Enthusiast,/l/Dragonchain/HmGO6thY4k/mcafee-indictment-an...,post,<p>It’s with sad news that I share with you al...,150293a5efa64a0ab86448fa59a062f3,Crypto_Enthusiast,95.64,0,False,35e0c07f244c484db4df398f7a323b08,Dragonchain,,1861,1861,0,8,28,1,10.0000,18610.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y A L L W E I R D,/l/Dragonchain/a9I6vlAlvG/march-2021-takara-pr...,reply,<p>How does this effect node rewards?</p>,b23ff2e0de924a06b5bde6a1a4c20bf7,Y A L L W E I R D,96.94,0,False,35e0c07f244c484db4df398f7a323b08,Dragonchain,,5,5,0,1,0,1,0.0001,0.000,0
Crypto_Enthusiast,/l/Dragonchain/a9I6vlAlvG/march-2021-takara-pr...,reply,<p>Thx for the info.</p>,150293a5efa64a0ab86448fa59a062f3,Crypto_Enthusiast,95.64,0,False,35e0c07f244c484db4df398f7a323b08,Dragonchain,,2,2,0,1,0,0,0.0001,0.000,0
Rock316,/l/Dragonchain/a9I6vlAlvG/march-2021-takara-pr...,reply,<p>Takara is a &#34;stable price&#34; system t...,53518899830a4d07b8ae04e2f1888fd9,Rock316,100.00,0,False,35e0c07f244c484db4df398f7a323b08,Dragonchain,,55,55,0,1,2,1,0.0001,0.005,0
Crypto_Enthusiast,/l/Dragonchain/a9I6vlAlvG/march-2021-takara-pr...,reply,<p>Does anyone know what impact this has on ma...,150293a5efa64a0ab86448fa59a062f3,Crypto_Enthusiast,95.64,0,False,35e0c07f244c484db4df398f7a323b08,Dragonchain,,3,3,0,1,2,1,0.0001,0.000,0


In [48]:
def comment_data(data):
    url = []
    contentType = []
    content = []
    author_id = []
    author_displayName = []
    author_siteRep = []
    author_lairRep = []
    author_isBot = []
    lair_id = []
    lair_name = []
    lair_displayName = []
    stats_score = []
    stats_upVoteScore = []
    stats_downVoteScore = []
    stats_upVoters = []
    stats_views = []
    stats_replyCount = []
    stats_trendingScoreMultiplier = []
    stats_weightedScore = []
    stats_mtrBoost = []

    for i in range(data['count']):
        url.append(data['contents'][i]['url'])
        contentType.append(data['contents'][i]['contentType'])

        #This gives the content of the comment
        content.append(data['contents'][i]['content'])

        #This gives information on the author
        author_id.append(data['contents'][i]['author']['id'])
        author_displayName.append(data['contents'][i]['author']['displayName'])
        author_siteRep.append(data['contents'][i]['author']['siteReputation'])
        author_lairRep.append(data['contents'][i]['author']['lairReputation'])
        author_isBot.append(data['contents'][i]['author']['isBot'])

        # This shows which lair the contents came from 
        lair_id.append(data['contents'][i]['lair']['id'])
        lair_name.append(data['contents'][i]['lair']['name'])
        lair_displayName.append(data['contents'][i]['lair']['displayName'])

        #This is all the stats of that individual conmment
        stats_score.append(data['contents'][i]['stats']['score'])
        stats_upVoteScore.append(data['contents'][i]['stats']['upVoteScore'])
        stats_downVoteScore.append(data['contents'][i]['stats']['downVoteScore'])
        stats_upVoters.append(data['contents'][i]['stats']['upVoters'])
        stats_views.append(data['contents'][i]['stats']['views'])
        stats_replyCount.append(data['contents'][i]['stats']['replyCount'])
        stats_trendingScoreMultiplier.append(data['contents'][i]['stats']['trendingScoreMultiplier'])
        stats_weightedScore.append(data['contents'][i]['stats']['weightedScore'])
        stats_mtrBoost.append(data['contents'][i]['stats']['mtrBoost'])

    #url,contentType,content,author_id,author_displayName,author_siteRep,author_lairRep,author_isBot,lair_id,lair_name,lair_displayName,stats_score,stats_upVoteScore,stats_downVoteScore,stats_upVoters,stats_views,stats_replyCount,stats_trendingScoreMultiplier,stats_weightedScore,stats_mtrBoost

    comment_data = pd.DataFrame(list(zip(url,contentType,content,author_id,author_displayName,
                                      author_siteRep,author_lairRep,author_isBot,lair_id,lair_name,
                                      lair_displayName,stats_score,stats_upVoteScore,stats_downVoteScore,
                                      stats_upVoters,stats_views,stats_replyCount,stats_trendingScoreMultiplier,
                                      stats_weightedScore,stats_mtrBoost)),
                             columns =['url','contentType','content','author_id','author_displayName',
                                       'author_siteRep','author_lairRep','author_isBot','lair_id',
                                       'lair_name','lair_displayName','stats_score','stats_upVoteScore',
                                       'stats_downVoteScore','stats_upVoters','stats_views','stats_replyCount',
                                       'stats_trendingScoreMultiplier','stats_weightedScore','stats_mtrBoost'])
    return comment_data

In [49]:
# Looking at the first five 'Name' entries  in the dataframe for lair_data
lair_data['Name'][0:5]

covid19                  covid19
Cryptocurrency    Cryptocurrency
TechNews                TechNews
CrowsNest              CrowsNest
ama                          ama
Name: Name, dtype: object

In [50]:
# #Now lets try to sort all the data in the same way for all of the different lairs
# #Now there is a dataframe of information for each lair
# #NOTE: This takes a long time to run all, it would be better to pull only the ones of interest.

# # LairName = dataframe data for that lair
# covid19 = comment_data(data = main_p(lair_data['ID'][0]))
# cryptocurrency = comment_data(data = main_p(lair_data['ID'][1]))
# technews = comment_data(data = main_p(lair_data['ID'][2]))
# crowsnest = comment_data(data = main_p(lair_data['ID'][3]))
# ama = comment_data(data = main_p(lair_data['ID'][4]))
# Thebitcoinstreetjournal = comment_data(data = main_p(lair_data['ID'][5]))
# smaugust = comment_data(data = main_p(lair_data['ID'][6]))
# dragonshoard = comment_data(data = main_p(lair_data['ID'][7]))
# dragonchain_official_it = comment_data(data = main_p(lair_data['ID'][8]))
# dragonchain_official_nl = comment_data(data = main_p(lair_data['ID'][9]))
# qanon = comment_data(data = main_p(lair_data['ID'][10]))
# dragonchain_official_kr = comment_data(data = main_p(lair_data['ID'][11]))
# memes = comment_data(data = main_p(lair_data['ID'][12]))
# welcome_to_the_matrix = comment_data(data = main_p(lair_data['ID'][13]))
# dragonchain_official_fr = comment_data(data = main_p(lair_data['ID'][14]))
# hello = comment_data(data = main_p(lair_data['ID'][15]))
# eternal_archive = comment_data(data = main_p(lair_data['ID'][16]))
# hash_name = comment_data(data = main_p(lair_data['ID'][17]))
# redpills = comment_data(data = main_p(lair_data['ID'][18]))
# ems = comment_data(data = main_p(lair_data['ID'][19]))
# dragonchain = comment_data(data = main_p(lair_data['ID'][20]))
# dragonchain_official_pt = comment_data(data = main_p(lair_data['ID'][21]))
# sports = comment_data(data = main_p(lair_data['ID'][22]))
# maker = comment_data(data = main_p(lair_data['ID'][23]))
# dance = comment_data(data = main_p(lair_data['ID'][24]))
# pets = comment_data(data = main_p(lair_data['ID'][25]))
# music = comment_data(data = main_p(lair_data['ID'][26]))
# dragonchain_official_es = comment_data(data = main_p(lair_data['ID'][27]))
# christianity = comment_data(data = main_p(lair_data['ID'][28]))
# creaturecomforts = comment_data(data = main_p(lair_data['ID'][29]))
# politics = comment_data(data = main_p(lair_data['ID'][30]))
# DragonDen = comment_data(data = main_p(lair_data['ID'][31]))
# gov2021 = comment_data(data = main_p(lair_data['ID'][32]))
# code = comment_data(data = main_p(lair_data['ID'][33]))
# bored = comment_data(data = main_p(lair_data['ID'][34]))
# FourTwenty = comment_data(data = main_p(lair_data['ID'][35]))
# DenSupport = comment_data(data = main_p(lair_data['ID'][36]))
# art = comment_data(data = main_p(lair_data['ID'][37]))
# ISLM = comment_data(data = main_p(lair_data['ID'][38]))
# DrgnES = comment_data(data = main_p(lair_data['ID'][39]))
# gaming = comment_data(data = main_p(lair_data['ID'][40]))
# beaxy = comment_data(data = main_p(lair_data['ID'][41]))
# FIT = comment_data(data = main_p(lair_data['ID'][42]))
# TIL = comment_data(data = main_p(lair_data['ID'][43]))
# LCP = comment_data(data = main_p(lair_data['ID'][44]))
# lbty = comment_data(data = main_p(lair_data['ID'][45]))
# outdoors = comment_data(data = main_p(lair_data['ID'][46]))
# hackernoon = comment_data(data = main_p(lair_data['ID'][47]))
# dragonchain_official_jp = comment_data(data = main_p(lair_data['ID'][48]))



In [78]:
# This is the Comment DataFrame for Dragonchain Japanese
dragonchain_official_jp = comment_data(data = main_p(lair_data['ID'][48]))
dragonchain_official_jp

,url,contentType,content,author_id,author_displayName,author_siteRep,author_lairRep,author_isBot,lair_id,lair_name,lair_displayName,stats_score,stats_upVoteScore,stats_downVoteScore,stats_upVoters,stats_views,stats_replyCount,stats_trendingScoreMultiplier,stats_weightedScore,stats_mtrBoost
0,/l/dragonchain-official-jp/KIkfSQ2SsV/ドラゴンチェーン...,post,"<h1 id=""dragonchainawardedpatentforenterprises...",60f26780071d4ba385c0db5b6912ef31,YMDRGN,99.13,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,1645,1645,0,4,26,0,0.5000,822.500,0
1,/l/dragonchain-official-jp/ikeqYVTf5U/super-ha...,post,"<h3 id=""2021223"">2021年2月23日</h3>\n<h3 id=""82"">...",60f26780071d4ba385c0db5b6912ef31,YMDRGN,99.13,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,1509,1509,0,4,5,0,0.5000,754.500,0
2,/l/dragonchain-official-jp/G0DGsIWNDu/beaxyはdr...,post,"<h1 id=""beaxydragonchainchain"">BeaxyはDragoncha...",20e5881fe2864a6a9da0a9422bcd007a,FreezaDRGN,99.47,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,2084,2084,0,4,10,0,0.0025,5.210,0
3,/l/dragonchain-official-jp/B3hhnEgXhg/ブロックチェーン...,post,<p>世界中のサプライチェーンは、私たちのインフラの中で最も脆弱な部分の一つです。サイバー攻...,60f26780071d4ba385c0db5b6912ef31,YMDRGN,99.13,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,2173,2173,0,3,5,0,0.0001,0.217,0
4,/l/dragonchain-official-jp/x92Zmmxxgl/super-ha...,reply,<p>🐲🐲🐲</p>,42f5b479896f47f88cd5e4d8816877df,DrgnJohn,97.51,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,144,144,0,1,0,0,0.0001,0.014,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,/l/dragonchain-official-jp/OhDprUSuXO/super-ha...,post,<p>史上最高のコマーシャル！#Don’tSkipTheCommercial</p>\n<p...,20e5881fe2864a6a9da0a9422bcd007a,FreezaDRGN,99.47,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,1000,1000,0,1,16,0,0.0001,0.100,0
81,/l/dragonchain-official-jp/ca9c53275f7f4c2391a...,post,<p>2020年7月2日</p>\n<h2>議題</h2>\n<ul>\n<li>最初の手が...,20e5881fe2864a6a9da0a9422bcd007a,FreezaDRGN,99.47,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,1000,1000,0,1,11,0,0.0001,0.100,0
82,/l/dragonchain-official-jp/2d46ccabd88144aea09...,reply,<p>訂正</p>\n<p>btc = <br/>\neth = <br/>\n </p>...,20e5881fe2864a6a9da0a9422bcd007a,FreezaDRGN,99.47,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,0,0,0,0,0,0,0.0001,0.000,0
83,/l/dragonchain-official-jp/2d46ccabd88144aea09...,post,"<p>祭典では、<a href=""https://www.google.com/url?q=...",20e5881fe2864a6a9da0a9422bcd007a,FreezaDRGN,99.47,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,1466,1466,0,2,16,1,0.0001,0.147,0


In [83]:
# Now we can grab all the comments we can from a user, here we grbbed used information from FreezaDRGN
freezaDRGN = dragonchain_official_jp.loc[dragonchain_official_jp['author_displayName'] == 'FreezaDRGN']
freezaDRGN

,url,contentType,content,author_displayName,author_siteRep,author_lairRep,author_isBot,lair_id,lair_name,lair_displayName,stats_score,stats_upVoteScore,stats_downVoteScore,stats_upVoters,stats_views,stats_replyCount,stats_trendingScoreMultiplier,stats_weightedScore,stats_mtrBoost
author_id,,,,,,,,,,,,,,,,,,,
20e5881fe2864a6a9da0a9422bcd007a,/l/dragonchain-official-jp/G0DGsIWNDu/beaxyはdr...,post,"<h1 id=""beaxydragonchainchain"">BeaxyはDragoncha...",FreezaDRGN,99.47,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,2084,2084,0,4,10,0,0.0025,5.210,0
20e5881fe2864a6a9da0a9422bcd007a,/l/dragonchain-official-jp/x92Zmmxxgl/super-ha...,post,"<h1 id=""superhappydragonluckyepisode0x50recap""...",FreezaDRGN,99.47,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,2730,2730,0,4,11,1,0.0001,0.273,0
20e5881fe2864a6a9da0a9422bcd007a,/l/dragonchain-official-jp/fWbTb2cfOt/dragonch...,post,"<h1 id=""dragonchaingovernmentblockchainassocia...",FreezaDRGN,99.47,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,1425,1425,0,3,19,0,0.0001,0.142,0
20e5881fe2864a6a9da0a9422bcd007a,/l/dragonchain-official-jp/6HR70PKC2h/aqed-dra...,post,"<h1 id=""aqeddragonchainquantumsafeblockchain"">...",FreezaDRGN,99.47,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,2517,2517,0,4,18,0,0.0001,0.252,0
20e5881fe2864a6a9da0a9422bcd007a,/l/dragonchain-official-jp/MyVAYAEpEX/super-ha...,post,"<h2 id=""superhappydragonluckyepisode0x4erecap""...",FreezaDRGN,99.47,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,4605,4605,0,4,13,0,0.0001,0.460,0
20e5881fe2864a6a9da0a9422bcd007a,/l/dragonchain-official-jp/koL72r2Kp4/super-ha...,post,"<h1 id=""superhappydragonluckyepisode0x4crecap""...",FreezaDRGN,99.47,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,4259,4259,0,4,13,1,0.0001,0.426,0
20e5881fe2864a6a9da0a9422bcd007a,/l/dragonchain-official-jp/WorcFJf3Ip/den-gove...,post,"<p>Den GovernanceLOTを取得する方法</p>\n<p><img src=""...",FreezaDRGN,99.47,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,2058,2058,0,3,19,0,0.0001,0.206,0
20e5881fe2864a6a9da0a9422bcd007a,/l/dragonchain-official-jp/OIIwyjyHrr/super-ha...,post,"<h1 id=""superhappydragonluckyepisode0x4arecap""...",FreezaDRGN,99.47,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,2975,2975,0,2,10,0,0.0001,0.297,0
20e5881fe2864a6a9da0a9422bcd007a,/l/dragonchain-official-jp/hjHq73tTwc/異種のサイロ化さ...,reply,<p>You are welcome</p>,FreezaDRGN,99.47,0,False,9792e80365e54da5b32728943e0c636d,dragonchain-official-jp,公式ドラゴンチェーン日本語,234,234,0,2,0,0,0.0001,0.023,0


In [87]:
#Lets see how many times he made a post and how many times he made a reply
numPost = freezaDRGN.loc[freezaDRGN['contentType'] == 'post']
numReply = freezaDRGN.loc[freezaDRGN['contentType'] == 'reply']
len(numPost),len(numReply)

(31, 2)

#### Here we can see that he posted 32 time nd responded only 2 times, we can use this information later after we run sentiment analysis on how he is responding to others, whether he/she is more negative or positive

In [ ]:
# This is the Comment DataFrame for Christianity
christianity = comment_data(data = main_p(lair_data['ID'][28]))
christianity

In [ ]:
#Lets looks at the first comment in the christianity lair
christianity['content'][0]

In [ ]:
smaugust = comment_data(data = main_p(lair_data['ID'][6]))
smaugust

In [ ]:
df =pd.read_csv('user_ids and their posts.csv')

Joes posts:

In [ ]:
for i in df['cab6d19d22ec4bdfad4f99760d33312b']:
    print(i)